# Primary Notebook

## Purpose

The purpose of this notebook is the creation of the clusters.
Those (landscape) clusters are supposed to be areas similar in appearance;
To be more precise, these clusters share a common "FFT footprint" which will be elaborated on later.

## Other parts of this project

There are secondary notebooks, that might have to be run beforehand, to receive data which this primary notebook will work upon.

In [ ]:
# What we might need eventually
# rasterio for geotiffs
# dask array for parallel computing of large arrays
# pyfftw for 2d fft 

## Overview of the classes and their interconnection
The subject of this notebook is to cluster different landscapes across multiple DEM-GeoTIFFs. 

### GeographicBounds
This is an object that saves West, South, East, North borders, as well as the projection, and maybe some other additional info.

### AugmentedDEM
This will be the "container" for all information regarding *one* specific DEM raster map.

### EmbeddingMap
This will be part of each AugmentedDEM. Here we will save the labels created by the clustering.




## How to make the GeoTIFF images usable by my algorithm:
It’s important to note, that the original DEM data is in arc seconds. To avoid skewed results it has to be compressed by cos(latitude) in width.  That way the metric distances (almost) resemble the distances pixel-wise

After that the images will be scaled to 1/4 (as of now), because processing becomes about 4^2 times faster.

Before processing, the data will have to be split up into quadratic tiles which are supposed to be equal in their length.

I guess we should combine the processes, namely selecting tiles of equal size and then reducing their size to a given size in pixels

In [ ]:
# Imports

# Filesystem, JSON
import os
import json
from io import BytesIO # To be able to route the API response to a file-like object that rasterio can use

# Rasterio for handling GeoTIFFs
import rasterio

# Math may not be missing
import math
import numpy as np

# Regex for updating filenames
import re

# For earth related numbers
from pyproj import Geod
geod = Geod(ellps="WGS84")

In [ ]:
# Define classes specific to this project 

class GeographicBounds:
    def __init__(self, west, south, east, north):
        self.west = west
        self.north = north
        self.east = east
        self.south = south

class AugmentedDEM:
    def __init__(self, dem_path):
        


## Code to be refactored and integrated:

In [ ]:
# Das hier muss noch ordentlich gemacht werden

num_tiles_y = temporary_settings["dem_size_height"] / internal_settings["fft"]["tile_size"]
print(num_tiles_y)

relative_size_y = 1 / num_tiles_y
print(relative_size_y)

rest_tiles_y = num_tiles_y % 1;
print(rest_tiles_y)

relative_start_y = (rest_tiles_y / 2) / num_tiles_y
print(relative_start_y)

relative_end_y = 1 - relative_start_y

firstrow_center = np.array([relative_start_y / 2])

lastrow_center = np.array([1 - (relative_start_y / 2)])

all_other_rows_center = np.linspace(relative_start_y + (relative_size_y / 2), relative_end_y - (relative_size_y / 2), math.floor(num_tiles_y), endpoint=True)

internal_borders_y = np.linspace(relative_start_y, relative_end_y, math.ceil(num_tiles_y))
all_borders_y = np.concatenate((np.array([0]),internal_borders_y,np.array([1])))


allrows_centers = np.concatenate((firstrow_center, all_other_rows_center, lastrow_center))

print(allrows_centers)


In [ ]:
# Testing
np.interp(all_borders_y,[0,1],[internal_settings["geo"]["north"],internal_settings["geo"]["south"]])

In [ ]:
# Testing
np.interp(allrows_centers,[0,1],[internal_settings["geo"]["north"],internal_settings["geo"]["south"]])